In [1]:
import sys
from common_imports import *

/home/gayeon43/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-12 16:42:22.730505: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Skipping import of cpp extensions due to incompatible torch version 2.8.0+cu128 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info
/home/gayeon43/anaconda3/envs/da/lib/python3.9/site-packages/tensorflow_hub/__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for re

- SQuAD
    - context에 대한 question과 answer 존재 / 답은 문서 내 특정 위치(span)에 존재 
    - SQuAD v1.1: 답 없는 질문 없음
    - SQuAD v2.0: 답 없는 질문 포함(모델이 답 없음 판단해야 함)

In [2]:
# SQuAD 데이터 형식
# - data: 문서 리스트
#   - title: 문서 제목
#   - paragraphs: 문서 내 단락 리스트
#     - context: 단락 텍스트
#     - qas: 단락 내 질문 리스트
#       - question: 질문 텍스트
#       - id: 질문 ID
#       - q_type: 질문 유형('ABBR': 0, 'DESC': 1, 'ENTY': 2,'HUM': 3, 'LOC': 4, 'NUM': 5)
#       - answers: 정답 리스트
#         - text: 정답 텍스트
#         - answer_start: 정답 시작 위치(인덱스)

train_squad_path = '../../data/squad/train-v1.1_classified.json'
with open(train_squad_path, 'r') as f:
    train_squad_data = json.load(f)['data']
    
    
paragraph_len = 0
qas_len = 0
for i in train_squad_data:
    paragraph_len += len(i['paragraphs'])
    for j in i['paragraphs']:
        qas_len += len(j['qas'])
print('데이터 개수: ', len(train_squad_data))
print('문맥  개수: ',paragraph_len)
print('질문-정답 쌍 개수: ',qas_len)

print('train_squad_data.keys(): ', train_squad_data[0].keys())
train_squad_data[0]

데이터 개수:  442
문맥  개수:  18896
질문-정답 쌍 개수:  87599
train_squad_data.keys():  dict_keys(['title', 'paragraphs'])


{'title': 'University_of_Notre_Dame',
 'paragraphs': [{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
   'qas': [{'answers': [{'answer_start': 515,
       'text': 'Saint Bernadette Soubirous'}],
     'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
     'id': '5733be284776f41900661182',
     'q_ty

- 1) CNN : 뉴스 기사 기반의 cloze 스타일 QA 데이터셋으로, 문장 내 빈칸을 채우는 형식의 질문이 많음.  

In [128]:
train_cnn_path = '../../data/cnn/cnn_train_classified.json'

with open(train_cnn_path, 'r') as f:
    train_cnn_data = json.load(f)['data']

paragraph_len = 0
qas_len = 0
for i in train_cnn_data:
    paragraph_len += len(i['paragraphs'])
    for j in i['paragraphs']:
        qas_len += len(j['qas'])
print('데이터 개수: ', len(train_cnn_data))
print('문맥 개수: ',paragraph_len)
print('질문-정답 쌍 개수: ',qas_len)

# print('first data paragraphs len: ', len(train_cnn_data[0]['paragraphs']))
# print('first data title: ', train_cnn_data[0]['title'])
# print('paragraphs data keys: ',train_cnn_data[0]['paragraphs'][0].keys())
# print('context example: ', train_cnn_data[1]['paragraphs'][0]['context'])
# print('question example: ', train_cnn_data[1]['paragraphs'][0]['qas'][0]['question'])
# print('answer example: ', train_cnn_data[1]['paragraphs'][0]['qas'][0]['answers'])

print('train_cnn_data.keys(): ', train_cnn_data[0].keys())
train_cnn_data[0]

데이터 개수:  93627
문맥 개수:  93627
질문-정답 쌍 개수:  93627
train_cnn_data.keys():  dict_keys(['title', 'paragraphs'])


{'title': 'CNN',
 'paragraphs': [{'context': '-- two of the most influential papers for voters in Iowa and New Hampshire -- the first two states to weigh in at the polls in 2008 -- both endorsed John McCain in the GOP presidential race , but differed in their choice in the Democratic contest . the Register backs Hillary Clinton , while the Globe picks Obama . the Register backed Hillary Clinton , while the Globe picked Obama , in excerpts of sunday \'s editorials posted on their papers \' web sites saturday night . the Iowa caucuses are january 3 , and New Hampshire \'s primary follows five days later . the Globe \'s editorial board dismissed concerns over the Illinois senator \'s relative lack of Washington experience . " it is true that all the other Democratic contenders have more conventional resumes , and have spent more time in Washington , " the board wrote . " but that exposure has tended to give them a sense of government \'s constraints . Obama is more open to its possibiliti

- 2) Daily Mail: CNN과 유사한 뉴스 기반 cloze QA 데이터셋으로 간단한 문장 매칭·복원 문제 비중이 큼.  

In [129]:
train_daily_path = '../../data/daily/daily_train_classified.json'

with open(train_daily_path, 'r') as f:
    train_daily_data = json.load(f)['data']

paragraph_len = 0
qas_len = 0
for i in train_daily_data:
    paragraph_len += len(i['paragraphs'])
    for j in i['paragraphs']:
        qas_len += len(j['qas'])
print('데이터 개수: ', len(train_daily_data))   
print('문맥 개수: ',paragraph_len)
print('질문-정답 쌍 개수: ',qas_len)

print('train_daily_data.keys(): ', train_daily_data[0].keys())
train_daily_data[0]

데이터 개수:  87253
문맥 개수:  87253
질문-정답 쌍 개수:  87253
train_daily_data.keys():  dict_keys(['title', 'paragraphs'])


{'title': 'Daily Mail',
 'paragraphs': [{'context': "by Jonny Singer Sunderland owner Short has upped the stakes in the North East rivalry by nominating three prominent personnel at Newcastle to take part in the Ice Bucket Challenge . Short posted a video of his own challenge on Sunderland 's youtube channel as centre - back pairing West Brown and John O'Shea douse him in water beside the Stadium of Light pitch . however the real entertainment in the short video comes not from the suited Short screaming as he is drenched , but from his choice of nominations . video : scroll down to watch Short nominate a Newcastle trio for the Ice Bucket Challenge short video : the Sunderland owner takes the challenge next to the Stadium of Light pitch ready to go : having nominated his rivals , Short prepares for the ice cold water drenched : John O'Shea and Wes Brown empty the bin of freezing water over their boss 's head all over : as players and coaches laugh , Short gets up from his Ice Bucket Cha

- 3) NewsQA: 뉴스 기사에서 발췌된 지문과 인간작성 질문을 포함한 QA 데이터셋으로, 단순 매칭을 넘어 추론이 요구됨.  

In [130]:
train_news_path = '../../data/news/news_train_classified.json'

with open(train_news_path, 'r') as f:
    train_news_data = json.load(f)['data']

paragraph_len = 0
qas_len = 0
for i in train_news_data:
    paragraph_len += len(i['paragraphs'])
    for j in i['paragraphs']:
        qas_len += len(j['qas'])
print('데이터 개수: ', len(train_news_data))   
print('문맥 개수: ',paragraph_len)
print('질문-정답 쌍 개수: ',qas_len)

print('train_news_data.keys(): ', train_news_data[0].keys())
train_news_data[0]

데이터 개수:  11448
문맥 개수:  11448
질문-정답 쌍 개수:  76340
train_news_data.keys():  dict_keys(['title', 'paragraphs'])


{'title': 'cnn',
 'paragraphs': [{'context': 'A high court in northern India on Friday acquitted a wealthy businessman facing the death sentence for the killing of a teen in a case dubbed "the house of horrors." Moninder Singh Pandher was sentenced to death by a lower court in February. The teen was one of 19 victims -- children and young women -- in one of the most gruesome serial killings in India in recent years. The Allahabad high court has acquitted Moninder Singh Pandher, his lawyer Sikandar B. Kochar told CNN. Pandher and his domestic employee Surinder Koli were sentenced to death in February by a lower court for the rape and murder of the 14-year-old. The high court upheld Koli\'s death sentence, Kochar said. The two were arrested two years ago after body parts packed in plastic bags were found near their home in Noida, a New Delhi suburb. Their home was later dubbed a "house of horrors" by the Indian media. Pandher was not named a main suspect by investigators initially, but w

- 4) HotQA : 여러 문서(또는 문단)를 연결해 답을 찾아야 하는 multi-hop(다중추론) QA 데이터셋.  

In [131]:
train_hotqa_path = '../../data/hotpot/hotpot_train_classified.jsonl'

train_hotqa_data = []
with open(train_hotqa_path, 'r') as f:
    for line in f:
        train_hotqa_data.append(json.loads(line))

qas_len = 0
for i in train_hotqa_data:
    qas_len += len(i['qas'])    
print('데이터, 문맥, 질문-쌍 개수: ', qas_len)
train_hotqa_data[0]

데이터, 문맥, 질문-쌍 개수:  72928


{'id': '5a879ab05542996e4f30887e',
 'context': '[PAR] [TLE] The Oberoi Group [SEP] The Oberoi Group is a hotel company with its head office in Delhi.  Founded in 1934, the company owns and/or operates 30+ luxury hotels and two river cruise ships in six countries, primarily under its Oberoi Hotels & Resorts and Trident Hotels brands. [PAR] [TLE] Oberoi family [SEP] The Oberoi family is an Indian family that is famous for its involvement in hotels, namely through The Oberoi Group. ',
 'qas': [{'id': '5a879ab05542996e4f30887e#0',
   'question': 'The Oberoi family is part of a hotel company that has a head office in what city?',
   'answers': ['Delhi'],
   'qid': '197390227efc4f86b313a2b585cd685b',
   'question_tokens': [['The', 0],
    ['Oberoi', 4],
    ['family', 11],
    ['is', 18],
    ['part', 21],
    ['of', 26],
    ['a', 29],
    ['hotel', 31],
    ['company', 37],
    ['that', 45],
    ['has', 50],
    ['a', 54],
    ['head', 56],
    ['office', 61],
    ['in', 68],
    ['what', 

- 5) SearchQA: 원래 QA 쿼리에 대해 검색엔진으로 수집한 컨텍스트를 붙여 만든 데이터셋으로 현실적 검색 결과 노이즈가 특징.  

In [132]:
train_search_path = '../../data/search/search_train_classified.jsonl'

train_search_data = []
with open(train_search_path, 'r') as f:
    for line in f:
        train_search_data.append(json.loads(line))
    
qas_len = 0
for i in train_search_data:
    qas_len += len(i['qas'])    
print('데이터, 문맥, 질문-쌍 개수: ', qas_len)
# train_search_data[0]

데이터, 문맥, 질문-쌍 개수:  117384


In [133]:
print('SQuAD keys:', train_squad_data[0].keys())
print('SQuAD paragraphs keys:', train_squad_data[0]['paragraphs'][0].keys())
print('SQuAD qas keys:', train_squad_data[0]['paragraphs'][0]['qas'][0].keys())
print('-'*100)
print('NewsQA keys:', train_news_data[0].keys())
print('NewsQA paragraphs keys:', train_news_data[0]['paragraphs'][0].keys())
print('NewsQA qas keys:', train_news_data[0]['paragraphs'][0]['qas'][0].keys())
print('-'*100)
print('CNN keys:', train_cnn_data[0].keys())
print('CNN paragraphs keys:', train_cnn_data[0]['paragraphs'][0].keys())
print('CNN qas keys:', train_cnn_data[0]['paragraphs'][0]['qas'][0].keys())
print('-'*100)
print('Daily Mail keys:', train_daily_data[0].keys())
print('Daily Mail paragraphs keys:', train_daily_data[0]['paragraphs'][0].keys())
print('Daily Mail qas keys:', train_daily_data[0]['paragraphs'][0]['qas'][0].keys())
print('-'*100)
print('SearchQA keys:', train_search_data[0].keys())
print('SearchQA qas keys:', train_search_data[0]['qas'][0].keys())
print('-'*100)
print('HotpotQA keys:', train_hotqa_data[0].keys())
print('HotpotQA qas keys:', train_hotqa_data[0]['qas'][0].keys())

SQuAD keys: dict_keys(['title', 'paragraphs'])
SQuAD paragraphs keys: dict_keys(['context', 'qas'])
SQuAD qas keys: dict_keys(['answers', 'question', 'id', 'q_type'])
----------------------------------------------------------------------------------------------------
NewsQA keys: dict_keys(['title', 'paragraphs'])
NewsQA paragraphs keys: dict_keys(['context', 'qas'])
NewsQA qas keys: dict_keys(['answers', 'question', 'id', 'q_type'])
----------------------------------------------------------------------------------------------------
CNN keys: dict_keys(['title', 'paragraphs'])
CNN paragraphs keys: dict_keys(['context', 'qas'])
CNN qas keys: dict_keys(['answers', 'question', 'id', 'q_type'])
----------------------------------------------------------------------------------------------------
Daily Mail keys: dict_keys(['title', 'paragraphs'])
Daily Mail paragraphs keys: dict_keys(['context', 'qas'])
Daily Mail qas keys: dict_keys(['answers', 'question', 'id', 'q_type'])
-----------------

In [ ]:
squad_path = '../../data/squad/train-v1.1_classified_qtype_prob.jsonl'
cnn_path = '../../data/cnn/cnn_train_classified_qtype_prob.jsonl'
hotpot_path = '../../data/hotpot/hotpot_train_classified_qtype_prob.jsonl'

def open_jsonl(path):
    """
    JSONL 파일 로드 함수
    """
    data = []
    with io.open(path, 'r', encoding='utf-8') as f:
        for example in f:
            data.append(json.loads(example))
    # data = data[:5] # debug size
    return data

def open_json(path):
    """
    JSON 파일 로드 함수
    """
    with io.open(path, 'r', encoding='utf-8') as f:
        input_data = json.load(f)["data"]
    # input_data = input_data[:5]  # debug size
    return input_data

squads = open_jsonl(squad_path)
cnns = open_jsonl(cnn_path)
hotpots = open_jsonl(hotpot_path)

print(squads[0].keys())
print(cnns[0].keys())
print(hotpots[0].keys())

In [3]:
def convert_jsonl_to_json(data):
    """
    jsonl 파일을 json 파일 형식으로 변환하는 함수
    """
    converted_data = []
    for item in data:
        # title: data의 id
        title = item.get('id', '')
        # context: data의 context
        context = item.get('context', '')
        # qas: data의 qas에서 필요한 필드만 추출
        qas = []
        for qa in item.get('qas', []):
            new_qa = {
                'answers': qa.get('answers', []),
                'question': qa.get('question', ''),
                'id': qa.get('qid', ''),  # qid를 id로
                'q_type': qa.get('q_type', None)
            }
            qas.append(new_qa)
        
        # jsonl 형식으로 구성
        new_entry = {
            'title': title,
            'paragraphs': [{
                'context': context,
                'qas': qas
            }]
        }
        converted_data.append(new_entry)
    return converted_data

def save_jsonl_to_json(input_path: str, output_path: str):
    """
    JSONL을 JSON 형식으로 변환하는 함수
    """
    data = open_jsonl(input_path)
    converted_data = convert_jsonl_to_json(data)
    json_data = {"data": converted_data}
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)
        
        
def clean_sentence(text: str) -> str:
    """
    문장 전처리 기능 함수
    축약형·소유격·하이픈 결합 단어 전처리
    """
    # 소유격, 축약형 's 분리
    text = re.sub(r"(\w+)'s", r"\1 's", text)
    # 축약형 단순 변환s
    text = text.replace("n't", " not")
    text = text.replace("'re", " are")
    text = text.replace("'ve", " have")
    text = text.replace("'ll", " will")
    text = text.replace("'d", " would")
    text = text.replace("'m", " am")
    text = text.replace("What's", "What is") 
    # 하이픈 단어 분리
    text = re.sub(r"(\w+)-(\w+)", r"\1 \2", text)
    # 불필요한 특수문자 제거
    text = re.sub(r"[^A-Za-z0-9\s\?]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def preprocess_sentences(sent_list):
    """
    문장 리스트 전처리 함수
    """
    return [clean_sentence(s) for s in sent_list]

def open_jsonl(path):
    """
    JSONL 파일 로드 함수
    """
    data = []
    with io.open(path, 'r', encoding='utf-8') as f:
        for example in f:
            data.append(json.loads(example))
    # data = data[:5] # debug size
    return data

def open_json(path):
    """
    JSON 파일 로드 함수
    """
    with io.open(path, 'r', encoding='utf-8') as f:
        input_data = json.load(f)["data"]
    # input_data = input_data[:5]  # debug size
    return input_data

def update_qtype_prob(fpath, q_type, q_type_prob, q_ids):
    # JSON 파일 로드
    with io.open(fpath, 'r', encoding='utf-8') as f:
        input_data = json.load(f)["data"]
        input_data = input_data[:5] ################ 개수 줄이기 ################
    # 각 질문에 대해 분류 결과를 추가
    total_idx = 0
    for entry in input_data:
        for paragraph in entry["paragraphs"]:
            for qa in paragraph['qas']:
                # 예측된 질문 유형과 유형 확률값을 해당 질문에 추가
                if qa['id'] == q_ids[total_idx]:
                    qa['q_type'] = q_type[total_idx]
                    qa['q_type_prob'] = q_type_prob[total_idx]
                    total_idx += 1
                else:
                    # 질문 ID가 맞지 않으면 경고 출력
                    print('Can not match qid:', q_ids[total_idx])
                    
def update_file(fpath, q_type, q_type_prob, q_ids):
    """
    기존 파일에 질문 유형(q_type)과 질문 유형 확률(q_type_prob)을 추가하는 함수
    * format은 squad 형식으로 통합 *
    """

    # JSON 파일 처리(Ex.SQuAD)
    with io.open(fpath, 'r', encoding='utf-8') as f:
        input_data = json.load(f)["data"]
        input_data = input_data[:5] ################ 개수 줄이기 ################
    # 각 질문에 대해 분류 결과를 추가
    total_idx = 0
    for entry in input_data:
        for paragraph in entry["paragraphs"]:
            for qa in paragraph['qas']:
                # 예측된 질문 유형과 유형 확률값을 해당 질문에 추가
                if qa['id'] == q_ids[total_idx]:
                    qa['q_type'] = q_type[total_idx]
                    qa['q_type_prob'] = q_type_prob[total_idx]
                    total_idx += 1
                else:
                    # 질문 ID가 맞지 않으면 경고 출력
                    print('Can not match qid:', q_ids[total_idx])
        
    # 새로운 파일로 저장 (원본 파일명.json->_qtype_prob.jsonl)
    with open(fpath[:-5]+'_qtype_prob.jsonl', 'w') as f:
        for sample in input_data:
            f.write(json.dumps(sample)+'\n')
    f.close()
        

def parse_custom_data(raw_data):
    formatted_data = []
    for article in raw_data:
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                # 우리가 필요한 건 context, q_type -> question 입니다.
                formatted_data.append({
                    'context': context.strip(),
                    'q_type': qa['q_type'],
                    'question': qa['question'].strip()
                })
    return formatted_data

In [4]:
squad_path = '../../data/squad/train-v1.1_classified_qtype_prob.jsonl'
squad = open_jsonl(squad_path)
squad[0]

{'title': 'University_of_Notre_Dame',
 'paragraphs': [{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
   'qas': [{'answers': [{'answer_start': 515,
       'text': 'Saint Bernadette Soubirous'}],
     'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
     'id': '5733be284776f41900661182',
     'q_ty

In [ ]:
# jsonl(SearchQA) -> json
search_jsonl_path = '../../data/search/search_train_classified.jsonl'
search_json_path = '../../data/search/search_train_classified.json'
save_jsonl_to_json(search_jsonl_path, search_json_path)

In [ ]:
# jsonl(HotpotQA) -> json
hotpot_jsonl_path = '../../data/hotpot/hotpot_train_classified.jsonl'
hotpot_json_path = '../../data/hotpot/hotpot_train_classified.json'
save_jsonl_to_json(hotpot_jsonl_path, hotpot_json_path)

In [12]:
squad_dev_path = '../../data/squad/dev-v1.1.json'
squad_dev = open_json(squad_dev_path)
squad_dev[0]

{'title': 'Super_Bowl_50',
 'paragraphs': [{'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
   'qas': [{'answers': [{'answer_start': 177, 'text': 'Denver Broncos'},
      {'answer_start': 177, 'text': 'Denver Broncos'},
      {'answer_start': 177, 'text': '

In [10]:
cnn_dev_path = '../../data/cnn/cnn_dev.json'
cnn_dev = open_json(cnn_dev_path)
cnn_dev[0]

{'title': 'CNN',
 'paragraphs': [{'context': 'at Kathmandu \'s Tribhuvan International Airport , hundreds of migrant workers line up daily at the immigration counters , clutching newly printed passports and boarding passes . many of them , though , are clueless as to where they \'re headed . as a frequent traveler to the country , i \'ve been approached many times by these men , asking me to help fill out their departure forms . usually , when i ask what country they \'re traveling to , i get the same answer : " i \'m not sure . " they seem weary and lost but still hopeful that they can make their lives better . this time , however , i \'m at the airport to follow up on the repatriation of the most unfortunate of these migrant workers -- far too many of them make the return journey in caskets . almost every day , the remains of three or four workers arrive back in Nepal from the Middle East , according to Bhola Prasad Siwakoti , the secretary of the Nepal Ministry of Labor and Manpower

In [17]:
hotpot_dev_path = '../../data/hotpot/hotpot_dev.jsonl'
hotpot_dev = open_jsonl(hotpot_dev_path)
hotpot_dev[0]

{'header': {'dataset': 'HotpotQA', 'split': 'dev'}}

In [19]:
hotpot_dev[0]['header']

{'dataset': 'HotpotQA', 'split': 'dev'}

In [26]:
hotpot_dev[0]['header']['dataset']

'HotpotQA'

In [16]:
search_dev_path = '../../data/search/search_dev.jsonl'
search_dev = open_jsonl(search_dev_path)
search_dev[0]

{'header': {'dataset': 'SearchQA', 'split': 'dev'}}

In [6]:
new_hotpot = open_json(hotpot_json_path)
new_hotpot[0]

{'title': '5a879ab05542996e4f30887e',
 'paragraphs': [{'context': '[PAR] [TLE] The Oberoi Group [SEP] The Oberoi Group is a hotel company with its head office in Delhi.  Founded in 1934, the company owns and/or operates 30+ luxury hotels and two river cruise ships in six countries, primarily under its Oberoi Hotels & Resorts and Trident Hotels brands. [PAR] [TLE] Oberoi family [SEP] The Oberoi family is an Indian family that is famous for its involvement in hotels, namely through The Oberoi Group. ',
   'qas': [{'answers': ['Delhi'],
     'question': 'The Oberoi family is part of a hotel company that has a head office in what city?',
     'id': '197390227efc4f86b313a2b585cd685b',
     'q_type': 3}]}]}

In [144]:
new_search = open_json(search_json_path)
new_search[0]

{'title': '19',
 'paragraphs': [{'context': '[DOC] [TLE] I BROKED IT by outofculture  Pull Request #1  petekinnecom ... [PAR] + "question": "\'No. 10: FB/LB for Columbia U. in the 1920s; MVP for the Yankees \nin \'27 & \'36; \\"Gibraltar in Cleats\\"\'",. + "value": "$800",. + "answer": "(Lou) Gehrig... [DOC] [TLE] Jeopardy! #1 Flashcards | Quizlet [PAR] ESPN\'s TOP 10 ALL-TIME ATHLETES 800 No. 10: FB/LB for Columbia U. in the \n1920s; MVP for the Yankees in \'27 & \'36; "Gibraltar in Cleats". (Lou) Gehrig. ',
   'qas': [{'answers': ['(Lou) Gehrig'],
     'question': 'No. 10: FB/LB for Columbia U. in the 1920s; MVP for the Yankees in \'27 & \'36; "Gibraltar in Cleats"',
     'id': '7381db7eeccd4aff8290a391fd87fb28',
     'q_type': 3}]}]}

In [ ]:
hotpot = open_jsonl(hotpot_jsonl_path)

In [26]:
paragraph = hotpot[0]
qas = paragraph['qas'][0]
qas['id']

'5a879ab05542996e4f30887e#0'

In [48]:
fpath = hotpot_file_path
data = []
qa_data = []
qa_ids = []
with io.open(fpath, 'r', encoding='utf-8') as f:
    for example in f:
        data.append(json.loads(example))
data = data[:5]

for entry in data:
    for paragraph in entry["paragraphs"]:
        for qa in paragraph['qas']:
            qa_data.append(qa['question'])
            qa_ids.append(qa['id'])

In [35]:
hotpot_file_path = '../../data/hotpot/hotpot_train_gen_question.jsonl'

def return_qa_qa_ids(fpath):
    """
    데이터 파일을 읽어 질문과 질문 ID를 추출
    tgt2idx = {'ABBR': 0, 'DESC': 1, 'ENTY': 2,'HUM': 3, 'LOC': 4, 'NUM': 5}
    """
    qa_data = []
    qa_ids = []

    if fpath.endswith('.jsonl'):
    # JSONL 파일 처리(Ex.HotPot)
        data = []
        with io.open(fpath, 'r', encoding='utf-8') as f:
            for example in f:
                data.append(json.loads(example))
        data = data[:5] ################ 개수 줄이기 ################
        for paragraph in data:
            for qas in paragraph['qas']:
                qa_data.append(qas['question'])
                qa_ids.append(qas['id'])
        return qa_data, qa_ids     
      
    elif fpath.endswith('.json'):
        # JSON 파일 처리(Ex.SQuAD, CNN)
        with io.open(fpath, 'r', encoding='utf-8') as f:
            data = json.load(f)["data"]
            data = data[:5] ################ 개수 줄이기 ################
        for entry in data:
            for paragraph in entry["paragraphs"]:
                for qa in paragraph["qas"]:
                    qa_data.append(qa['question'])
                    qa_ids.append(qa['id'])
        return qa_data, qa_ids
qa_data = []
qa_ids = []
all_qs, all_q_ids = return_qa_qa_ids(hotpot_file_path)

KeyError: 'qas'